Aqui está um código simples com sklearn para testar a execução de Python no Codespace com uma Decision Tree:

In [97]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [89]:
def carregar_dataset(dataset_func):
    """
    Descrição:
        Essa função é usada para carregar um dataset do Scikit-learn, separando-o em dados e rótulos.

    Parâmetros:
        dataset_func: função
            Função para carregar o dataset (por exemplo, `load_iris`, `load_digits`).

    Retorno:
        dataset: objeto
            O dataset completo carregado.
        X: array
            As características (dados) do dataset.
        y: array
            Os rótulos do dataset.
    """
    # Carregar o dataset especifico
    dataset = dataset_func()
    X = dataset.data
    Y = dataset.target

    return dataset, X, Y

In [90]:
def dividir_treinamento(X, Y, test_size=0.2, random_state=42):
    """"
    Descrição: 
        Função usada para Dividir matrizes ou matrizes em subconjuntos de trem aleatório e teste. para isso vou usar a função train_test_split()
        
    Parametrôs: data, label, test_size, random_state
        X = Características de entrada
        Y = Rotulos
        test_size = representar a proporção do conjunto de dados para incluir na divisão de teste.
        random_state = Controla o embaralhamento aplicado aos dados antes de aplicar a divisão. Passe um int para saída reprodutível em várias chamadas de função.

    Retorno: X_train, X_test, y_train, y_test
        X_train = dados de treinamento separado
        X_test = dados de teste separado
        y_train = rotulos de treinamento
        y_test = rotulos de teste
    """
    # Dividir o dataset em treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test

In [91]:
def normalizar_dados(X_train, X_test):
    """
    Descrição:
        Função para normalizar os dados de treinamento e teste usando StandardScaler.
    
    Parâmetros:
        X_train: array-like
            Dados de treinamento.
        X_test: array-like
            Dados de teste.
    
    Retorno:
        X_train_normalizado: array-like
            Dados de treinamento normalizados.
        X_test_normalizado: array-like
            Dados de teste normalizados.
        scaler: StandardScaler
            O objeto StandardScaler treinado, que pode ser usado para normalizar novos dados no futuro.
    """
    # Instanciando o StandardScaler
    scaler = StandardScaler()
    
    # Ajustando o scaler e transformando os dados de treinamento
    X_train_normalizado = scaler.fit_transform(X_train)
    
    # Transformando os dados de teste com o mesmo scaler
    X_test_normalizado = scaler.transform(X_test)
    
    return X_train_normalizado, X_test_normalizado, scaler

In [77]:
def treinando_modelo(modelo, X_train, y_train, X_test, random_state=None):
    """
    Descrição:
        Função para treinar um modelo de classificação usando o sklearn.
    
    Parâmetros:
        modelo: classificador
            O modelo que queremos usar como classificador (por exemplo, DecisionTreeClassifier, RandomForestClassifier).
        X_train: array-like
            Dados de treinamento.
        X_test: array-like
            Dados de teste.
        y_train: array-like
            Rótulos do conjunto de treinamento.
        random_state: int, opcional
            Controla o embaralhamento aplicado aos dados antes de aplicar a divisão. Passe um int para saída reprodutível.

    Retorno:
        modelo_treinado: classificador
            O modelo treinado.
        y_pred: array
            Previsões do modelo para o conjunto de teste.
    """
    # Verificar se o modelo aceita o parâmetro random_state
    if 'random_state' in modelo().get_params().keys():
        modelo_treinado = modelo(random_state=random_state)
    else:
        modelo_treinado = modelo()

    # Treinar o modelo
    modelo_treinado.fit(X_train, y_train)

    # Prever as classes para o conjunto de teste
    y_pred = modelo_treinado.predict(X_test)

    return modelo_treinado, y_pred

In [83]:
def treinando_com_grid_search(modelo, param_grid, X_train, y_train, X_test, cv=5, random_state=42):
    """
    Descrição:
        Função para treinar um modelo de classificação usando GridSearchCV para encontrar os melhores hiperparâmetros.
    
    Parâmetros:
        modelo: classificador
            O modelo que queremos usar como classificador (por exemplo, DecisionTreeClassifier, RandomForestClassifier).
        param_grid: dict
            Dicionário contendo os hiperparâmetros que serão testados no GridSearchCV.
        X_train: array-like
            Dados de treinamento.
        y_train: array-like
            Rótulos do conjunto de treinamento.
        X_test: array-like
            Dados de teste.
        cv: int, opcional (default=5)
            Número de divisões para a validação cruzada.
        random_state: int, opcional (default=42)
            Controla o embaralhamento aplicado aos dados para garantir reprodutibilidade.

    Retorno:
        melhor_modelo: classificador
            O melhor modelo treinado com os hiperparâmetros otimizados.
        y_pred: array
            Previsões do modelo para o conjunto de teste.
        melhores_parametros: dict
            Os melhores hiperparâmetros encontrados pelo GridSearchCV.
    """
    # Instanciando o GridSearchCV
    grid_search = GridSearchCV(estimator=modelo(random_state=random_state), param_grid=param_grid, cv=cv, n_jobs=-1, verbose=2)
    
    # Executando o GridSearchCV para encontrar os melhores hiperparâmetros
    grid_search.fit(X_train, y_train)
    
    # Obter os melhores parâmetros
    melhores_parametros = grid_search.best_params_
    
    # Treinar o modelo final com os melhores parâmetros
    melhor_modelo = grid_search.best_estimator_
    
    # Prever as classes para o conjunto de teste
    y_pred = melhor_modelo.predict(X_test)
    
    return melhor_modelo, y_pred, melhores_parametros

In [85]:
def configurar_e_treinar_modelo_arvore_decisao(X_train, y_train, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None, random_state=42):
    """
    Descrição:
        Função para configurar e treinar um modelo de árvore de decisão usando o conjunto de dados de treinamento.

    Parâmetros:
        X_train: array-like
            Dados de treinamento.
        y_train: array-like
            Rótulos do conjunto de treinamento.
        max_depth: int ou None, opcional (default=None)
            Profundidade máxima da árvore. Se None, os nós são expandidos até todas as folhas serem puras.
        min_samples_split: int, opcional (default=2)
            O número mínimo de amostras necessárias para dividir um nó.
        min_samples_leaf: int, opcional (default=1)
            O número mínimo de amostras necessárias para estar em um nó folha.
        max_features: int, float, string ou None, opcional (default=None)
            O número de características a considerar ao procurar a melhor divisão.
        random_state: int, opcional (default=42)
            Controla o embaralhamento aplicado aos dados para garantir reprodutibilidade.

    Retorno:
        modelo_treinado: classificador
            O modelo treinado de árvore de decisão.
    """
    # Configurando o modelo de árvore de decisão
    modelo = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=random_state
    )

    # Treinando o modelo
    modelo_treinado = modelo.fit(X_train, y_train)

    return modelo_treinado

In [92]:
dataset, X, Y = carregar_dataset(load_iris)
# Agora, você pode usar `X` e `y` em outras partes do seu código
print("Características (X):", X[:5])
print("Rótulos (y):", Y[:5])

X_train, X_test, y_train, y_test = dividir_treinamento(X, Y)

X_train_normalizado, X_test_normalizado, scaler = normalizar_dados(X_train, X_test)

modelo_treinado, y_pred = treinando_modelo(DecisionTreeClassifier, X_train_normalizado, y_train, X_test_normalizado, random_state=42)
print("Precisão:", modelo_treinado.score(X_test, y_test))

Características (X): [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
Rótulos (y): [0 0 0 0 0]
Precisão: 0.36666666666666664


In [94]:
# Definindo o espaço de hiperparâmetros para o GridSearchCV
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Chamando a função treinando_com_grid_search
melhor_modelo, y_pred, melhores_parametros = treinando_com_grid_search(DecisionTreeClassifier, param_grid, X_train_normalizado, y_train, X_test_normalizado)

# Exibindo os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:", melhores_parametros)

# Exibindo o relatório de classificação
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))
print("Precisão:", melhor_modelo.score(X_test_normalizado, y_test))

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1

In [96]:
# Configurando e treinando o modelo com parâmetros específicos
modelo_treinado = configurar_e_treinar_modelo_arvore_decisao(X_train_normalizado, y_train, max_depth=10, min_samples_split=5, min_samples_leaf=2, max_features='sqrt')

# Agora você pode usar o modelo treinado para fazer previsões
y_pred = modelo_treinado.predict(X_test_normalizado)

# Avaliando o desempenho do modelo
from sklearn.metrics import classification_report
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred))

Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.89      0.94         9
           2       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30

